# **Funciones**

Este notebook está basado en el curso de [_Julia Academy_](https://github.com/JuliaAcademy/Introduction-to-Julia).

Temas:
1. Como declarar una función en Julia
2. "Duck-typing" en Julia
3. Funciones mutantes vs no mutantes en Julia
4. Algunas funciones de mayor orden

Una función es un bloque de código que podemos llamar para ser ejecutado en nuestro programa y ejecutar una tarea específica. Un ejemplo que hemos utilizado ya por ejemplo es la función `print`, que imprime en pantalla los argumentos que le demos. Aprenderemos ahora a escribir nuestras propias funciones y algunas características útiles que tienen en `Julia`.

## **Como declarar una función en Julia**
Julia tiene varias formas de declarar funciones. La primera es utilizando las palabras clave `function` y `end`.

In [ ]:
function saludo(nombre)
    println("Hola $nombre, ¡qué bueno verte!")
end

`saludo` toma un argumento `nombre`, y llama a la función `print` para escribir un saludo.

In [ ]:
function f(x)
    x^2
end
#Nota: podríamos haber escrito return x^2, y también habría funcionado
#Las funciones de Julia retornan el resultado de la última operación que
#hicieron.

`f` toma un argumento `x`, y devuelve el resultado de la operación `x^2`.

Podemos llamar a estas funciones de la siguiente manera:

In [ ]:
saludo("Mario")

In [ ]:
f(49)

También podemos definir funciones en una sola línea si es que son suficientemente cortas.

In [ ]:
salu2(nombre) = println("Hola $nombre, ¡qué bueno verte!")

In [ ]:
f2(x) = x^2

Tanto `salu2` como `f2` son idénticas a `saludo` y `f`. Sólo las hemos definido de una forma más compacta.

In [ ]:
salu2("María")

In [ ]:
f2(42)

Finalmente, podríamos haberlas declarado como funciones "anónimas"

In [ ]:
saludo3 = nombre -> println("Hola $nombre, ¡qué bueno verte!")

In [ ]:
f3 = x -> x^2

In [ ]:
saludo3("René")

In [ ]:
f3(42)

El uso principal de las funciones anónimas es para dárselas a otras funciones que tomen funciones como argumento, y veremos ejemplos más adelante. Por el momento no se preocupe mucho por ellas.

Para definir funciones que acepten más argumentos, simplemente debemos escribir los argumentos en el orden que serán aceptados.

In [ ]:
function maximo(x,y)
    return x > y ? x : y
    #===============================================================================
    Nota: aquí hemos utilizado un operador ternario. Esto es equivalente a haber
    escrito
    
    if x > y
        return x
    else
        return y
    end
    ================================================================================#
    
end
println(maximo("abc","ajk"))
println(maximo(2, 8))

## **Duck-typing in Julia**
*"Si hace cuak como un pato, es un pato."* <br><br>
Las funciones de Julia van a funcionar siempre que las operaciones estén bien definidas para sus argumentos. <br><br>
Por ejemplo, la función `saludo` que definimos anteriormente funciona si el argumento es un número

In [ ]:
saludo(3.14)

Y `f` funcionará en una matriz. 

In [ ]:
A = rand(3, 3) #Esta línea crea una matriz de 3x3 con elementos aleatorios entre 0 y 1
A

In [ ]:
f(A)  #f(A) = A^2 = A*A. Notar que la operación que realizó Julia fue la multiplicación
      #de matrices, no la multiplicación elemento por elemento.

`f` también va a funcionar con un `string`, ya que `^` es el operador utilizado para repetir strings.

In [ ]:
f("hola")  #"hola"^2 = "hola"*"hola" = "holahola"

Sin embargo, `f` no va a funcionar en un vector. Esto es porque no podemos multiplicar vectores columna por vectores columna.

In [ ]:
v = rand(3)

In [ ]:
# Esta línea dará error, ya que v es un vector columna, y v*v no está bien definido.
f(v)

## **Funciones mutantes vs. no-mutantes**

A esta altura puede que haya notado que algunas funciones en los tutoriales anteriores tienen un signo `!`. Por convención, las funciones que terminan en un signo `!` pueden alterar el contenido de sus argumentos, y las que no lo tienen no.

Por ejemplo, miremos la diferencia entre las funciones de ordenamiento `sort` y `sort!`

In [ ]:
v = [3, 5, 2] #Creamos un vector v

In [ ]:
sort(v)      #Vemos qué retorna la función sort(v)

In [ ]:
v            #Comprobamos que v no fue modificado

`sort(v)` retorna un arreglo ordenado de los elementos de `v`, pero `v` no cambia. <br><br>

Por otro lado, si usamos `sort!(v)`, los contenidos de `v` son ordenados dentro del mismo arreglo.

In [ ]:
sort!(v)    #Vemos qué retorna la función sort!(v)

In [ ]:
v

**Importante**
Es una buena práctica usar esta convención al definir funciones, de manera de saber rápido si la función que está utilizando corre el riesgo de modificar vectores que ha definido. 

## **Comportamiento de las funciones ante arreglos y diccionarios** ###

Veamos qué sucede si intentamos modificar una variable dentro de una función:

In [ ]:
function duplicar(x)
    x = 2*x
end
a = 2
b = duplicar(a)
println("a = $a, b = $b")

Notamos que `a` no fue modificada por la función `duplicar`, pese a que internamente realizamos la operación `x = 2*x`. Esto es porque `duplicar` utilizó internamente una copia de `a`, y por lo tanto no modificó la variable original.

Por el contrario, cuando una función recibe como argumento una estructura de datos mutable, puede modificarla. Por ejemplo, la siguiente función duplica los elementos de un arreglo:

In [ ]:
function duplicar!(arreglo)
    for i in 1:size(arreglo,1) #El "1" indica que nos interesa el tamaño en la primer dimensión del arreglo
        arreglo[i] *= 2
        #=======================================================================
        arreglo[i] *= 2 es una forma compacta de escribir
        arreglo[i] = arreglo[i]*2
        Esta notación también funciona para otras operaciones como +, - y /
        =======================================================================#
    end
    return arreglo #Esta línea no es necesaria, solo la agregamos para que
                   #la función retorne algo
end

In [ ]:
duplicar!([1,2,3])

In [ ]:
v = [1,2,3]
duplicar!(v)
print(v)

Como somos buenas personas y no queremos confundir a nadie, agregamos un signo `!`, avisando que nuestra función modificará el argumento de entrada que le demos.
Si no tiene nada mejor que hacer en este momento, pruebe crear una función `duplicar` que retorne el un arreglo multiplicado por 2 sin modificarlo.

También podemos modificar elementos de un diccionario. Por ejemplo, la siguiente función agrega una entrada a un diccionario:

In [ ]:
function agregar_numero!(diccionario, contacto, numero)
    diccionario[contacto] = numero #agregamos un contacto a un diccionario
end
agenda = Dict("Juan" => 5551234, "Martina" => 5558954)
agregar_numero!(agenda, "Sofía", 5559812)
agenda

Esto es muy útil, ya que Julia no necesita crear copias de los arreglos para trabajar dentro de las funciones, pero es también importante tenerlo en cuenta para no cometer errores y modificar un arreglo cuando no queremos hacerlo.

## **Algunas funciones de mayor orden**

### `map`

`map` es una función de "mayor orden" de Julia que *toma una función* como uno de sus argumentos. `map` luego aplica esa función a todos los elementos de una estructura de datos que le pasemos.
Por ejemplo, ejecutar

```julia
map(f, [1, 2, 3])
```
retornará un arreglo donde `f` fue aplicada a todos los elementos de `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [ ]:
map(f, [1, 2, 3])   #Recordemos que más arriba definimos f(x) = x^2

Es decir, elevamos al cuadrado cada elemento de `[1, 2, 3]`.

También podríamos haberle dado a `map` una función anónima,

In [ ]:
map(x -> x^3, [1, 2, 3]) #x -> x^3 es una función anónima, es decir, no tiene nombre.
                         #La definimos únicamente para pasarla como argumento a la
                         #función map

y ahora hemos elevado al cubo los elementos de `[1, 2, 3]`

### `broadcast`

`broadcast` es otra función de mayor orden como `map`. `broadcast` es una generaliación de `map`, por lo que puede hacer lo mismo que `map` y mucho más. La sintaxis básica de `broadcast` es la misma que para `map`

In [ ]:
broadcast(f, [1, 2, 3])

Y nuevamente aplicamos `f` a todos los elementos de `[1, 2, 3]`

`broadcast` se diferencia de `map` en cómo se comporta ante multiples argumentos de distintas dimensiones. `broadcast` intentará encontrar una dimensión en común y `map` no. Por ejemplo, notar la diferencia entre estas dos líneas:

In [ ]:
map(+, 1, [1,2,3]) #Nota: Los operadores de Julia son funciones. 
                   #      Escribir +(a,b) es lo mismo que escribir a+b.

In [ ]:
broadcast(+,1,[1,2,3])

Una abreviatura muy útil de `broadcast` es colocar `.` entre una función a la que quiera hacer `broadcast` y sus argumentos. Por ejemplo,

```julia
broadcast(f, [1, 2, 3])
```
es equivalente a
```julia
f.([1, 2, 3])
```

In [ ]:
f.([1, 2, 3])

Note nuevamente como esto es distinto a llamar
```julia
f([1, 2, 3])
```
Podemos elevar al cuadrado cada elemento del vector, ¡pero esto no es lo mismo que elevar al cuadrado el vector, ya que esto no está definido!

Para mostrar más claramente la diferencia, considere

```julia
f(A)
```
y
```julia
f.(A)
```
para una matriz `A`:

In [ ]:
A = [i + 3*j for j in 0:2, i in 1:3]

In [ ]:
f(A)

En este caso estamos haciendo
```
f(A) = A^2 = A * A
``` 

Por otro lado,

In [ ]:
B = f.(A)

contiene los cuadrados de cada una de las entradas individuales de `A`.

Esta sintaxis nos permite escribir expresiones complejas de una forma mucho más natural

In [ ]:
A .+ 2 .* f.(A) ./ A

en lugar de

In [ ]:
broadcast(x -> x + 2 * f(x) / x, A)

y ambas expresiones van a tener la misma performance.

In [ ]:
@. A + 2 * f(A) / A

## **Especificando tipos**

Al definir una función, podemos especificar el `tipo` de argumentos que recibe. Incluso, como veremos más adelante en multiple dispatch, podemos hacer que una función se comporte distinto dependiendo del `tipo` de los argumentos que recibe. La sintaxis para especificar que una variable `a` tiene tipo `Type` es `a::Type`.
En el siguiente ejemplo definimos una función que únicamente funciona si el argumento es un número entero

In [ ]:
function factorial(n::Int)
    if n < 1
        println("Error, n < 1. Intente con un número positivo.")
        return -1
    end
    result = 1
    for i in 2:n
        result *= i
    end
    return result
end

In [ ]:
factorial(3)

In [ ]:
soloenteros(3.0)

## **Sobre los argumentos de las funciones**

### **Cantidad arbitraria de variables de argumentos**

La función `print` tiene un comportamiento interesante que quizás haya notado. Podemos darle una cantidad arbitraria de argumentos, y `print` imprime cada uno de ellos:

In [ ]:
print("hola", " yo", " estoy", " dándole", " muchos", " argumentos", " a", " print")

Esto se logra utilizando el operador _splat_. Aprenderemos de el creando nuestra propia función con una cantidad _variable_ de argumentos. La función `suma_arbitraria` suma una cantidad arbitraria de argumentos:

In [ ]:
function suma_arbitraria(x...)
    println("Argumentos dados: $x")
    return sum(x)
end

In [ ]:
suma_arbitraria(1,2,3,4)

Los elementos de "x" fueron leidos como una tupla. Podríamos tambien haber dado como argumento una tupla o un arreglo:

In [ ]:
suma_arbitraria([i for i in 1:10]...)

Note que si buscamos la documentación de `print`, el segundo argumento de la función es `xs...`.

In [ ]:
?print()

### **Argumentos opcionales**
Podemos dar valores por defecto a ciertas variables, lo cual puede ayudar a quien use la función a evitar dar argumentos innecesarios. La siguiente función retorna la fecha en el formato `(día, mes, año)`, y asume que es enero del año 2000 si no especificamos mes ni año: 

In [ ]:
function fecha(dd, mm = 1, aa = 2000)
    return (dd, mm, aa)
end

In [ ]:
fecha()    #Esto no funcionará, porque debemos especificar el día

In [ ]:
fecha(10) #Aquí, dd = 10, y mm = 1, aa = 2000 por defecto

In [ ]:
fecha(10,3) #Aquí hemos especificado el mes, pero no el año

In [ ]:
fecha(10,3,2022) #Finalmente, aquí especificamos tanto el día como el mes y el año

In [ ]:
fecha((10,3)...)  #Como dato extra, también podemos usar el operador ... para ingresar
                  #una tupla como argumentos separados en cualquier función

### **Argumentos Keyword** (Palabras clave) 

Un problema con la función que definimos anteriormente, es que debemos escribir los argumentos en el orden específico que los espera la función. No solo eso, sino que podemos ahorrarnos escribir el mes y no escribir el año, pero no podemos escribir el año e ignorar el mes. Para solucionar este problema, podemos usar `keyword arguments` tras un signo `;`.

In [ ]:
function fecha_v2(dd; mm = 1, aa = 2000, format = :dma)
    if format == :dma
        return (dd, mm, aa)
    elseif format == :mda
        return (mm, dd, aa)
    elseif format == :amd
        return (aa, mm, dd)
    else
        println("Formato de fecha no reconocido. Solo acepto dma, mda y amd")
    end
end

In [ ]:
fecha_v2(1,format = :mda, mm = 3) #El ; solo lo usamos para definir la función, no para llamarla

Ahora podemos escribir solo los argumentos que nos interesen y en el orden que querramos, aunque debemos especificar el nombre del argumento.

### **Último truco: cantidad arbitraria de keywords**

También podemos agregar una cantidad arbitraria de variables keywords utilizando splat. En este caso, la función guardará las variables en un diccionario. Veamos un ejemplo:

In [ ]:
function funcion_kwargs(argumento_1;x...)
    println("el primer argumento vale = $argumento_1")
    println("Usted ingresó los siguientes argumentos keyword")
    for (key, val) in x
        println("$key = $val")
    end
end

perro = 3

funcion_kwargs(1; a = 3, b = "manzana", perro)

#Note que cualquier cosa luego de un ; será considerado parte de los kwargs!
#La línea anterior sin ; no funciona

## **Cómo agregar documentación**
Para documentar una función, simplemente debemos agregar un string antes de la declaración de la función.

In [ ]:
"""
    funciondocumentada(x...)

# Ejemplo de documentación

Esta función existe únicamente para explicar cómo agregar documentación.
Escribir documentación funciona como escribir en modo **markdown**.

Note que para separar líneas debe poner una línea vacía entre medio.

Puede llamarla utilizando `funciondocumentada(argumentos)`.\n

Acepta una cantidad arbitraria de argumentos.
"""
function funciondocumentada(x...)
    pass
end

In [ ]:
@doc funciondocumentada

In [ ]:
using JLD2

In [ ]:
@doc jldsave

## **Ejercicios opcionales**
***1)*** Escriba una función `suma_uno` que sume 1 a su argumento

In [ ]:
@assert suma_uno(1) == 2

In [ ]:
@assert suma_uno(11) == 12

***2)*** Use `map` o `broadcast` para incrementar cada elemento de una matriz `A` en `1` y asígnelo a una variable `A1`.

In [ ]:
@assert A1 == [2 3 4; 5 6 7; 8 9 10]

***3)***
Use la sintaxis `.` de `broadcast` para incrementar todos los valores de `A1` en `1` y guarde el resultado en `A2`

In [ ]:
@assert A2 == [3 4 5; 6 7 8; 9 10 11]

***4)*** Cree una función `misuma(vec_1, vec_2)` que retorne la suma directa de `vec_1, vec_2`. Use para esto un loop `for` dentro de la función.
Luego cree otra función llamada `misuma!(vec_r, vec_1, vec_2)` que funcione igual, pero guarde el resultado en `vec_r`.

In [ ]:
#= 
function misuma(vec_1, vec_2)
    *código*
end
=#

In [ ]:
#= 
function misuma!(vec_r, vec_1, vec_2)
    *código*
end
=#

In [ ]:
#Verificación de misuma. Si esta celda no da error, misuma funciona.
vec_1 = [1,2,3]
vec_2 = [3,2,1]

for elemento in misuma(vec_1, vec_2)
    @assert elemento == 4
end

In [ ]:
#Verificación de misuma!. Si esta celda no da error, misuma! funciona.
vec_1 = [1,2,3]
vec_2 = [3,2,1]
vec_r = copy(vec_1)
misuma!(vec_r, vec_1, vec_2)
for elemento in vec_r
    @assert elemento == 4
end

***5)*** Repita el ejercicio anterior, pero usando broadcast para escribir una función más compacta

In [ ]:
vec_1 = [1,2,3]
vec_2 = [3,2,1]

for elemento in misuma(vec_1, vec_2)
    @assert elemento == 4
end

In [ ]:
vec_1 = [1,2,3]
vec_2 = [3,2,1]
vec_r = copy(vec_1)
misuma!(vec_r, vec_1, vec_2)
for elemento in vec_r
    @assert elemento == 4
end